# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_CGMGPT',
 'MODEL_TASK': 'Model_CGMGPT',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared '
                   'drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace'}


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [8]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools

####################
TriggerCaseMethod = 'CGM5MinEntry'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'CGM5Min',
 'case_id_columns': ['PID', 'ObsDT'],
 'special_columns': ['PID', 'DT_s'],
 'convert_TriggerEvent_to_Caseset': <function CGM5MinEntry.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [10]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS
print(cohort_args)
print(TriggerCasePath)

df_case = pd.read_pickle(TriggerCasePath) # (1000)
df_case = df_case.sample(100, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 100
})


## [Step 2] RO: Record Observation & CasePhi


In [11]:
###########################
ROName_List = ['CGM5Min-Bf24H']
name_CasePhi = 'RecNum'
###########################

In [12]:
from recfldtkn.obsname import convert_RONameList_to_COName

CaseObsName = convert_RONameList_to_COName(ROName_List, name_CasePhi)
print(CaseObsName)

ro.CGM5Min-Bf24H_ct.RecNum


In [13]:
# part of fetch_caseobs_Phi_tools
CO_Folder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CO_Folder): os.makedirs(CO_Folder)
print(CO_Folder)

../_Data/2-Data_CaseObs/ro.CGM5Min-Bf24H_ct.RecNum


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [14]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1002840, 1002580, 1004670, 1003564, 1003011], 'ObsDT': [Timestamp('2021-12-07 13:50:00'), Timestamp('2021-11-10 12:15:00'), Timestamp('2021-11-13 06:40:00'), Timestamp('2021-09-15 13:35:00'), Timestamp('2021-10-17 17:15:00')]}


In [15]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [16]:
case_example = idx_to_examples[0]
case_example

{'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}

## [Step 2] get_caseobs_id

In [17]:
# InObs - Rx - RecNum = 10

# CO_id
# CO: RO_RecNUm
# CO: PZipInfo COid # PID
# CO^{recnum}, CO^{futeduegm}

# {'PID': 1020788,
#  'ObsDT': Timestamp('2023-09-06 17:41:01.757000'),
#  'PInvID': '1020788-000',
#  'RxID': '1020788-000-000'} # <--- edu future engagement

# {'PID': 1020788,
#  'ObsDT': Timestamp('2023-09-06 17:41:01.757000'),
#  'PInvID': '1020788-000',
#  'RxID': '1020788-000-001'} # <--- edu future engagement

In [18]:
import inspect
#################################################
def get_CO_id(case_example, CaseObsName):
    PIDValue = case_example['PID']
    ObsDTValue = case_example['ObsDT'].isoformat()
    return f'{PIDValue}&{ObsDTValue}'

get_CO_id.fn_string = inspect.getsource(get_CO_id)
#################################################

In [19]:
caseobs_id = get_CO_id(case_example, CaseObsName)
caseobs_id # CO_id

'1002840&2021-12-07T13:50:00'



## [Step 3] RO List and CasePhi

In [20]:
name_CasePhi 

'RecNum'

In [21]:
## we need to prepare these five arguments.
# CaseTkn     # <------- we have this.
# cohort_args # <------- we have this.

# RecObs_Name 
# rec_args
# column_names # attribute name of R^recordname

In [22]:
######### RecObs_Name
print(ROName_List) # RO_List
# Let's focus on one RO
ROName = ROName_List[0]
print(ROName)

['CGM5Min-Bf24H']
CGM5Min-Bf24H


In [23]:
# RO = 'Rx-Bf2M-InsCate'
# RO = 'Rx-Bf2M'
# RO = 'P-Zip3DemoNume'

## [Step4] RO Information

In [24]:
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(ROName)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

CGM5Min Bf24H None


In [25]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
[i for i in rec_args]

CGM5Min


['CohortInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'DTCols',
 'RecDT',
 'ParentRecName',
 'attr_cols',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'pypath',
 'recfldtkn_config_path',
 'yaml_file_path']

## [Step 5] Load ds_rec

In [26]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)
# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [27]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PatientID',
 'CGM5MinID',
 'DT_s',
 'BGValue',
 'CGM5Min-N2Cin1Tkn_wgt',
 'CGM5Min-N2Cin1Tkn_tknidx']

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [28]:
###############################
def get_selected_columns(RecObs_Name, column_names, cohort_args, rec_args, CaseTkn):
    # RecName = RecObs_Name.split('-')[0]
    base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
    return base_columns

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [29]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?
get_selected_columns(ROName, column_names, cohort_args, rec_args, name_CasePhi)

['PID', 'CGM5MinID', 'DT_s']

In [30]:
print(ROName_List)
print(name_CasePhi)
print(get_selected_columns)

['CGM5Min-Bf24H']
RecNum
<function get_selected_columns at 0x7fdcb10a7700>


In [31]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
ROName_to_ROInfo = get_RecObsName_to_RecObsInfo(ROName_List, 
                                                name_CasePhi, 
                                                get_selected_columns,
                                                cohort_args, 
                                                cohort_args['Ckpd_ObservationS'], 
                                                record_to_ds_rec, 
                                                record_to_ds_rec_info)
    
ROName = ROName_List[0] # Rx-Bf2M  
ROInfo = ROName_to_ROInfo[ROName] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)

In [32]:
[i for i in ROInfo]
# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'fldtkn_args',
 'fld_tokenizer_fn',
 'run_fldtkn_on_the_fly',
 'ds_rec',
 'df_rec_info']

In [33]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 21215912
})

## [Step 7] get_CO_vocab

In [34]:
import inspect 

######################################
def get_CO_vocab(ROName_to_ROInfo):
    # phi_list
    RecFldName_list = list(set([RecObsInfo['RecName'] + '-' +RecObsInfo['FldName'] 
                                for RecObsName, RecObsInfo in ROName_to_ROInfo.items()
                                if RecObsInfo['FldName'] is not None ]
                                ))
    
    # print('RecFldName_list----->', RecFldName_list)
    # for current version: for any Phi, we only process one or zero phi. 
    assert len(RecFldName_list) <= 1

    ############################
    idx2tkn = ['[UNK]', 'recnum', 'recspan']
    ############################

    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CO_vocab = {}
    CO_vocab['tid'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    CO_vocab['wgt'] = {}
    # SeqType_List = ['tid', 'wgt']
    return CO_vocab
######################################

get_CO_vocab.fn_string = inspect.getsource(get_CO_vocab)


In [35]:
CO_vocab = get_CO_vocab(ROName_to_ROInfo)
pprint(CO_vocab, sort_dicts=False)

{'tid': {'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recspan'},
         'tkn2tid': {'[UNK]': 0, 'recnum': 1, 'recspan': 2}},
 'wgt': {}}



## [Step 8] Fetch ds_rec

In [36]:
case_example = idx_to_examples[4]
case_example

{'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}

In [37]:
ROName

'CGM5Min-Bf24H'

In [38]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [39]:
for ROName, ROInfo in ROName_to_ROInfo.items():
    print(ROName, ':', [i for i in ROInfo])

CGM5Min-Bf24H : ['rec_args', 'RecName', 'CkpdName', 'CkpdInfo', 'FldName', 'FldTknName', 'FldIdx2Tkn', 'fldtkn_args', 'fld_tokenizer_fn', 'run_fldtkn_on_the_fly', 'ds_rec', 'df_rec_info']


In [40]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import CaseObserverTransformer

get_Record_P = CaseObserverTransformer.get_Record_P
RecName_to_REC_P = get_Record_P(idx_to_examples, ROName_to_ROInfo)
for RecName, REC_P in RecName_to_REC_P.items():
    print(f'============ {RecName} ============')
    pprint(REC_P)

============ CGM5Min ============
{1002580: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 35212
}),
 1002840: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 47367
}),
 1003011: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 51200
}),
 1003564: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 39938
}),
 1004670: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 20889
})}


## [Step 9] Fetch ROdf: $R_i^{recname}$

In [41]:
get_idx_to_RecObsName_to_RecObsDS = CaseObserverTransformer.get_idx_to_RecObsName_to_RecObsDS
idx_to_ROName_to_ROds = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, ROName_to_ROInfo, RecName_to_REC_P)

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    print('ROdf:', idx_to_ROName_to_ROds[idx]) # R_i^{RecNum, Ckpd}
    print('====================')


idx: 0
case: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}
ROdf: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}

idx: 1
case: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}
ROdf: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}

idx: 2
case: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')}
ROdf: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 252
})}

idx: 3
case: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')}
ROdf: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}

idx: 4
case: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}
ROdf: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}


In [42]:
[i for i in ROName_to_ROInfo]

['CGM5Min-Bf24H']

In [43]:
CO_vocab

{'tid': {'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recspan'},
  'tkn2tid': {'[UNK]': 0, 'recnum': 1, 'recspan': 2}},
 'wgt': {}}

In [44]:
idx = 1
case_example = idx_to_examples[idx]
print('case_example:', case_example)
ROName_to_ROds = idx_to_ROName_to_ROds[idx] # RO_to_ROdf # RO_ij
print('RO_to_ROdf (RO_ij):', ROName_to_ROds)
print([i for i in ROName_to_ROds])

case_example: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}
RO_to_ROdf (RO_ij): {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}
['CGM5Min-Bf24H']


# [Step 10]: Develop $\Phi$. 

In [45]:
def fn_CasePhi(case_example, 
               ROName_to_ROds, 
               ROName_to_ROInfo, 
               CO_vocab):
    
    
    d = {}
    assert len(ROName_to_ROds) == 1
 
    # RO
    RO = list(ROName_to_ROds.keys())[0]
    # for RecObsName in RecObsName_to_RecObsDS:
    
    #############################################
    ROds   = ROName_to_ROds[RO] # dataframe: RecObsName: Rx-bf24.. RecObsDS: the df: record collection
    ROInfo = ROName_to_ROInfo[RO]
    
    RecDT = ROInfo['rec_args']['RecDT']
    if ROds is not None:
        DT_s_obs = ROds[ 0][RecDT] # the time of first records
        DT_e_obs = ROds[-1][RecDT] # pd.to_datetime(dates[idx_e-1]) # the last one smaller than idx_e
        d['recnum'] = len(ROds)
        d['recspan'] = (DT_e_obs - DT_s_obs).total_seconds() / 60 # + 5
    else:
        d['recnum'] = 0
        d['recspan'] = 0
    #############################################

    tkn2tid = CO_vocab['tid']['tkn2tid']
    tkn = [i for i in d.keys()]
    tid = [tkn2tid[i] for i in tkn]
    wgt = [round(d[i], 2) for i in tkn]
    CO = {'tid': tid, 'wgt': wgt}
    return CO

fn_CasePhi.fn_string = inspect.getsource(fn_CasePhi)

In [46]:
ROName_to_ROds

{'CGM5Min-Bf24H': Dataset({
     features: ['PID', 'CGM5MinID', 'DT_s'],
     num_rows: 289
 })}

In [47]:
fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab)

{'tid': [1, 2], 'wgt': [289, 1440.0]}

In [48]:
24 * 60

1440

In [49]:
CO_vocab

{'tid': {'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recspan'},
  'tkn2tid': {'[UNK]': 0, 'recnum': 1, 'recspan': 2}},
 'wgt': {}}

## Save Files

In [50]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CO_id, get_selected_columns, get_CO_vocab, fn_CasePhi]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casephi', f'phi_{name_CasePhi}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [51]:
CaseObsName

'ro.CGM5Min-Bf24H_ct.RecNum'

In [52]:
from recfldtkn.observer import get_CaseObsInfo_for_a_CaseObsName

CaseObsInfo = get_CaseObsInfo_for_a_CaseObsName(CaseObsName,
                                                SPACE, 
                                                cohort_args, 
                                                record_to_ds_rec = {}, 
                                                record_to_ds_rec_info = {})
[i for i in CaseObsInfo]

['RecObsName_List',
 'name_CasePhi',
 'get_selected_columns',
 'CaseObsName',
 'ROName_to_ROInfo',
 'fn_CasePhi',
 'get_CO_id',
 'CO_Folder',
 'CO_vocab']

In [54]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


ro_to_ROName = {} # leave it empty for now 
ROName_to_ROInfo = CaseObsInfo['ROName_to_ROInfo']
name_CasePhi = CaseObsInfo['name_CasePhi']
fn_CasePhi = CaseObsInfo['fn_CasePhi']
CO_vocab = CaseObsInfo['CO_vocab']
get_CO_id = CaseObsInfo['get_CO_id']
CO_Folder = CaseObsInfo['CO_Folder']
df_case = None 
use_CO_from_disk = False
batch_size = CaseObsInfo.get('batch_size', 1000)

fn_caseobs_Phi = CaseObserverTransformer(ro_to_ROName,
                                         ROName_to_ROInfo, 
                                         name_CasePhi, 
                                         fn_CasePhi, 
                                         CO_vocab, 
                                         get_CO_id,
                                         CO_Folder, 
                                         df_case,
                                         use_CO_from_disk)

# Check Cache Functions

In [55]:
fn_caseobs_Phi.new_COs

{}

In [56]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.380090
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt'],
    num_rows: 100
})


In [57]:
print(len(fn_caseobs_Phi.new_COs))

100


In [58]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.022781
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt'],
    num_rows: 100
})


In [59]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CO_vocab)

ro.CGM5Min-Bf24H_ct.RecNum
28
{'ObsDT': Timestamp('2023-05-20 17:35:00'),
 'PID': 2000020,
 'tid': [1, 2],
 'wgt': [289.0, 6305.0]}
{'tid': {'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recspan'}, 'tkn2tid': {'[UNK]': 0, 'recnum': 1, 'recspan': 2}}, 'wgt': {}}


In [61]:
ds_casetkn[62]

{'PID': 1004917,
 'ObsDT': Timestamp('2021-12-07 20:05:00'),
 'tid': [1, 2],
 'wgt': [289.0, 1440.0]}

## Save

In [62]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [63]:
CO_Folder_data = fn_caseobs_Phi.CO_Folder_data 
CO_Folder_data

'../_Data/2-Data_CaseObs/ro.CGM5Min-Bf24H_ct.RecNum/data'

In [64]:
fn_caseobs_Phi.save_new_COs_to_disk(CO_Folder_data)

[INFO:2024-03-03 01:16:55,328:(observer.py@630 recfldtkn.observer)]: use_CO_from_disk is false and empty new_COs


In [65]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [66]:
fn_caseobs_Phi.ds_CO_data

In [67]:
ds_CO_data, df_CO_info = fn_caseobs_Phi.load_COs_from_disk(CO_Folder_data, COids)

In [68]:
ds_CO_data

In [69]:
df_CO_info

,caseobs_idx_in_data
caseobs_id,


# Save Vocab

In [70]:
CO_vocab = fn_caseobs_Phi.CO_vocab
CO_vocab

{'tid': {'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recspan'},
  'tkn2tid': {'[UNK]': 0, 'recnum': 1, 'recspan': 2}},
 'wgt': {}}

In [71]:
CO_Folder_vocab = fn_caseobs_Phi.CO_Folder_vocab
CO_Folder_vocab

'../_Data/2-Data_CaseObs/ro.CGM5Min-Bf24H_ct.RecNum/vocab.p'

In [72]:
df_Vocab = pd.DataFrame({CaseObsName: CO_vocab})
df_Vocab.to_pickle(CO_Folder_vocab)
df_Vocab

,ro.CGM5Min-Bf24H_ct.RecNum
tid,"{'tid2tkn': {0: '[UNK]', 1: 'recnum', 2: 'recs..."
wgt,{}
